# Transform to Load into DDBB (2)

I need to transform the datasets to adapt and load them into a DDBB

0. Import modules
1. Load clean dataframes
2. Get a global view of the data to reorganize the DDBB tables
3. Reorganize and create new tables 
4. Save the new dataframes
5. Create a DDBB and load the DF's

### 0. Import modules

In [2]:
# Import modules etc

import pandas as pd
import numpy as np
import re
import random
from unidecode import unidecode
import re

np.random.seed(42)
pd.set_option('display.max_columns', None) # show all the columns

import warnings
warnings.filterwarnings('ignore') # ignorar warnings

from src import dataanalysis_fun1 as mlg # Import my module

####
# import mysql.connector as conn
# from sqlalchemy import create_engine

## 1. Load clean dataframes

In [4]:
kaggle_clean = pd.read_csv('../data/clean/kaggle_clean.csv')
asos_womenupperclothes_clean = pd.read_csv('../data/clean/asos_womenupperclothes_clean.csv')
amz1_womentshirt_clean = pd.read_csv('../data/clean/amz1_womentshirt_clean.csv')

In [5]:
asos_womenupperclothes_clean["shop"]="asos"
asos_womenupperclothes_clean.head()

,brand,description,price,colour,shop
0,New Look,Top negro escalonado de manga larga con estamp...,16.99,negro,asos
1,Mamalicious,Top color ámbar dorado de manga larga con cuel...,45.99,dorado,asos
2,Mamalicious,Top amarillo de manga larga con cuello ancho d...,21.99,dorado,asos
3,Selected,Top marrón de manga larga con cuello alto de S...,31.99,marron,asos
4,Mamalicious,Top multicolor a rayas con cuello alto de punt...,40.99,multicolor,asos


In [6]:
amz1_womentshirt_clean["shop"]="amazon"
amz1_womentshirt_clean.head()

,brand,description,price,colour,shop
0,Ranphee,Camisetas Túnica de Manga Larga para Mujer Blu...,25.99,negro,amazon
1,ANGGREK,Mujer Verano Tops Casual Encaje Cuello en V Ca...,31.19,negro,amazon
2,GRMLRPT,2023 Camiseta de Manga 3/4 Mujer Moda Cuello e...,25.99,negro,amazon
3,Urban Classics,Ladies Extended Shoulder tee Camiseta para Mujer,11.90,negro,amazon
4,Only,"Printed 3/4 Sleeved Top Suéter, Nero (Black/AO...",19.99,negro,amazon


## 2. Create 3 DFs `product`,  `costumer` and `sale`

In [7]:
product_df = pd.concat([asos_womenupperclothes_clean, amz1_womentshirt_clean], ignore_index=True)

In [8]:
product_df = product_df.sort_values(by='brand', ascending=True)
product_df["product_id"]=range(1,len(product_df)+1)
product_df.head()

,brand,description,price,colour,shop,product_id
9123,& Other Stories,Vestido corto rosa pastel con diseño bordado y...,54.0,rosa,asos,1
5153,& Other Stories,Vestido corto azul cruzado de mezcla de lino d...,43.5,azul,asos,2
5154,& Other Stories,Vestido midi blanco hueso con estampado de lun...,64.0,negro,asos,3
4741,& Other Stories,Vestido corto verde de corte skater con estamp...,45.0,verde,asos,4
7461,& Other Stories,Vestido corto con estampado a rayas de algodón...,36.5,multicolor,asos,5


### 2.1 Classify products according to their description


In [11]:
product_df1=product_df.copy()
product_df.head()

type_dict={"camiseta":["camiseta", "shirt", "polo", "Camicetta", "manga larga", "camisola", "manica lunga"],
           "top": ["top"],
           "chaqueta":["chaqueta","sudadera", "jersey", "pulover","pullover", "sueter", "polar", "cardigan", "casaca"],
           "vestido":["vestido", "tunica", "camison"],
           "camisa": ["camisa", "blusa", "blouse", "bluse"],
           "lenceria":["lenceria", "sujetador", "corse"],
           "chaleco": ["chaleco"],
          "casacas sanitarias": ["casacas sanitarias", "uniformes sanitarios", "uniforme sanitario", "trabajo"],
           "pantalones": ["pantalones",  "jeans", "vaqueros"],
          "mallas": [ "leggings", "mallas", "joggers", "mallot", "leotardo"],
            "falda": [ "falda"],
                       "zapatos": [ "zapatos"]
          }
                                                                          
def color_dict_simplify(x):
    x = x.lower()
    x=unidecode(x) #Take out accentuation
    theval=""
    for KKK, VVV in type_dict.items():
        for VAL in VVV:
            if VAL.lower() in x:
                theval = KKK
            
    return theval

product_df1["type"] = product_df1['description'].apply(color_dict_simplify)

print(len(product_df1[product_df1["type"]==""]))

134


#### Only 134 products remained without `type` classification

In [12]:
product_df1["type"].value_counts()

vestido               7192
camisa                6700
camiseta              6457
top                   2622
chaleco                648
casacas sanitarias     538
chaqueta               439
pantalones             143
falda                  139
lenceria               137
                       134
mallas                  29
zapatos                  6
Name: type, dtype: int64

### 2.2  Divide price of Packs

In [13]:
product_df2=product_df1.copy()

In [14]:
pack_dict={"pack":["pack", "packs"]}


def subset_pack(x):
    x = x.lower()
    x=unidecode(x)
    theval=1
    for KKK, VVV in pack_dict.items():
        for VAL in VVV:
            if VAL.lower() in x: # the item is actually a pack
                #find the number of elements and divide the price
                digits_only = re.sub(r'\D', '', x)
                numeric_values = [int(digit) for digit in digits_only]
                if numeric_values:
                    theval = min(numeric_values)
                else:
                    theval=1
            
    return theval

product_df2["price_pack"]=product_df2['description'].apply(subset_pack)
product_df2["price_pack"]=product_df2["price"]/product_df2["price_pack"]

In [15]:
product_complete=product_df2.reset_index(drop=True)
product_complete.index=product_complete.index+1
product_simple=product_complete[["brand", "colour", "shop", "product_id", "type", "price_pack"]]

## 3. Save `products` df

* `complete` info of products: including description and original price
* `simple` info of products: only brand, type, and price divided by pack #


The lines are commented to avoid overwriting

In [147]:
#product_simple.to_csv('../data/clean/product_simple.csv', index=False)
#product_complete.to_csv('../data/clean/product_complete.csv', index=False)

In [148]:
kaggle_clean.head()

,customer_id,age,gender,location,payment_method,review_rating,season
0,1,55,Male,teruel,Venmo,3.1,Winter
1,2,19,Male,cadiz,Cash,3.1,Winter
2,3,50,Male,barcelona,Credit Card,3.1,Spring
3,4,21,Male,ciudad_real,PayPal,3.5,Spring
4,5,45,Male,alava,PayPal,2.7,Spring


## 4. Create `customer_df` and `sale_df` from `kaggle_clean`

In [16]:
customer_df=kaggle_clean[["customer_id", "age", "gender", "location"]]

sale_df=kaggle_clean[["customer_id", "payment_method", "review_rating", "season"]]
sale_df["product_id"]=""

### 4.1 Add to `sale_df` randomly chosen values of `product_ids` from `product_df`

In [17]:
for i in range(len(sale_df)):
    sale_df["product_id"][i]=random.choice(product_simple.product_id)

#### There are multiple products absent in `sale_df` (i.e., not selled products)

It is expected, it is just an observation

In [24]:
AA=set(sale_df["product_id"]).difference(set(product_simple["product_id"]))
BB=set(product_simple["product_id"]).difference(set(sale_df["product_id"]))
print(len(AA), len(BB))

0 21570


#### All the customers have a sale_id!

In [23]:
AA=set(sale_df["customer_id"]).difference(set(customer_df["customer_id"]))
BB=set(customer_df["customer_id"]).difference(set(sale_df["customer_id"]))
print(len(AA), len(BB))

0 0


## 5.  `customer_df` and `sale_df`

The lines are commented to avoid overwriting

In [22]:
# customer_df.to_csv('../data/clean/customer_df.csv', index=False)
# sale_df.to_csv('../data/clean/sale_df.csv', index=False)

## XXXXXXX. Load and work with python - mySQL

In [244]:
from logs.PASSES import SQL ## my TOKEN
len(SQL)

6

### SqlAlchemy


* Create EER Diagram in mySQL workbench
* Load data (csv)

In [291]:
str_conn = 'mysql+pymysql://root:' + SQL + '@localhost:3306/mydb'
cursor = create_engine(str_conn)

In [292]:
product_df.to_sql(name='product',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)

In [293]:

customer_df.to_sql(name='customer',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)



### change the foreign key colnames

In [296]:
sale_df.head()
# Crea un diccionario para mapear nombres de columnas existentes a los nuevos nombres
nuevos_nombres = {'customer_id': 'customer_customer_id', 'payment_method': 'payment_method',
                 'review_rating': 'review_rating', 'season': 'season', 'product_id': 'product_product_id'}

# Utiliza la función rename para cambiar los nombres de las columnas
sale_df = sale_df.rename(columns=nuevos_nombres)
sale_df.head()

,customer_customer_id,payment_method,review_rating,season,product_product_id
0,1,Venmo,3.1,Winter,12993
1,2,Cash,3.1,Winter,5559
2,3,Credit Card,3.1,Spring,13271
3,4,PayPal,3.5,Spring,5251
4,5,PayPal,2.7,Spring,5287


In [297]:

sale_df.to_sql(name='sale',      # nombre de la tabla
            con=cursor,          # conexion al servidor
            if_exists='append',  # reemplaza la tabla si existe
            index=False)

